In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/data.rar > /dev/null

  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of gdrive/My Drive/data.rar or
        gdrive/My Drive/data.rar.zip, and cannot find gdrive/My Drive/data.rar.ZIP, period.


In [ ]:
!unrar x gdrive/My\ Drive/new_data.rar

In [ ]:
import os
import numpy as np
from tqdm import tqdm

In [ ]:
folder_names = [name for name in os.listdir('./') if name.startswith('dataset')]

In [ ]:
file_names = []
for folder in folder_names:
  for file in os.listdir(folder):
      file_names.append(os.path.join(folder,file))

In [ ]:
file_names[:5]

NameError: ignored

In [ ]:
print(len(file_names))

91069


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
min_value = 1000
max_value = 0

for file in tqdm(file_names):
    f = np.load(file)
    minv = np.amin(f[:,:,0])
    maxv = np.amax(f[:,:,0])
    if min_value>minv:
        min_value=minv
    if max_value<maxv:
        max_value=maxv

100%|██████████| 91069/91069 [01:32<00:00, 980.81it/s] 


In [ ]:
print(minv,maxv)

0.18311106 0.5188147


In [ ]:
def scaler(m):
    m_std = (m - minv) / (maxv - minv)
    m_std = np.expand_dims(m_std,axis=2)
    return m_std

In [ ]:
def inverse_scaler(m):
    m_scaled = m * (maxv - minv) + minv
    m_scaled = np.expand_dims(m_scaled,axis=2)
    return m_scaled

In [ ]:
def fix_missing_value(m):
  m[np.isnan(m)] = 0
  #m_fixed = np.expand_dims(m,axis=2)
  return m

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation, MaxPool2D, UpSampling2D
from tensorflow.keras.layers import BatchNormalization, Flatten, Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
optimizer = Adam(learning_rate=1e-6)

In [ ]:
lat = 128 # chunk size
long = 128 # chunk size

input_img = Input(shape=(lat, long,1))

x = Conv2D(64,(4,4), activation='relu', padding='same')(input_img)
x = Conv2D(128, (4, 4), activation='relu', padding='same')(x)
x = MaxPool2D((2, 2), padding='same')(x)
x = Conv2D(32, (4, 4), activation='relu', padding='same')(x)
x = MaxPool2D((2, 2), padding='same')(x)
x = Conv2D(16, (4, 4), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (4, 4), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)
# x = Conv2D(4, (4, 4), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(4, (4, 4), activation='relu',strides=(1,1), padding='same')(x)

x = Conv2DTranspose(8, (4, 4), strides=(1, 1),activation='relu', padding='same')(encoded)
x = Conv2DTranspose(16, (4, 4), strides=(2, 2),activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (4, 4), strides=(2, 2),activation='relu', padding='same')(x)
x = Conv2DTranspose(64, (4, 4), strides=(1, 1), activation='relu', padding='same')(x)
x = Conv2DTranspose(64, (4, 4), strides=(1, 1),activation='relu', padding='same')(x)
# x = Conv2DTranspose(128, (4, 4), strides=(2, 2),activation='relu', padding='same')(x)
decoded = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=optimizer, loss='mse')

In [ ]:
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 128, 128, 64)      1088      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 128, 128, 128)     131200    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 64, 64, 32)        65568     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 16)        8208

In [ ]:
file_paths = []
dir = './data_files'
for filename in os.listdir(dir):
    file_paths.append(dir+'/'+filename)

In [ ]:
len(file_paths)

69641

In [ ]:
np.random.seed(1)
np.random.shuffle(file_names)

files = {
    'training': file_names[:77410],
    'validation': file_names[77410:86517],
    'testing': file_names[86517:91069]
}

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Model'
    def __init__(self, files, batch_size=4, dim=(128,128,1), shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.files = files
        self.shuffle = shuffle
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.files) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        filenames = self.files[index*self.batch_size:(index+1)*self.batch_size]
        #shuffle
        if self.shuffle == True:
          np.random.shuffle(filenames)
        # Generate data
        X, y = self.__data_generation(filenames)

        return X, y

    def on_epoch_end(self):
        pass

    def __data_generation(self, filenames):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.load(filenames[0]) #X -> (128,128,1)
        X = fix_missing_value(X[:,:,:])
        X = scaler(X[:,:,0])
        X = np.expand_dims(X,axis=0) #X -> (1,128,128,1)
        # Generate data
        for file in filenames[1:]:
            # Store sample
            x = np.load(file) #X -> (128,128,1)
            x = fix_missing_value(x[:,:,:])
            x = scaler(x[:,:,0])
            x = np.expand_dims(x,axis=0) #X -> (1,128,128,1)
            X = np.append(X,x,axis=0)

        return X, X

In [ ]:
params = {'dim': (128,128,1),
          'batch_size':256,
          'shuffle': True}

params_v = {'dim': (128,128,1),
          'batch_size':256,
          'shuffle': True}

training_generator = DataGenerator(files['training'], **params)
validation_generator = DataGenerator(files['validation'], **params_v)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=1e-5, patience=5,restore_best_weights=True)

In [ ]:
history = autoencoder.fit(training_generator,epochs=50,
                    validation_data=validation_generator,shuffle=True,callbacks=[callback])

Epoch 1/50
302/302 [==============================] - 1144s 4s/step - loss: 0.2584 - val_loss: 0.2590
Epoch 2/50
302/302 [==============================] - 1105s 4s/step - loss: 0.2578 - val_loss: 0.2577
Epoch 3/50
302/302 [==============================] - 1105s 4s/step - loss: 0.2547 - val_loss: 0.2517
Epoch 4/50
302/302 [==============================] - 1110s 4s/step - loss: 0.2434 - val_loss: 0.2323
Epoch 5/50
302/302 [==============================] - 1111s 4s/step - loss: 0.2138 - val_loss: 0.1926
Epoch 6/50
302/302 [==============================] - 1198s 4s/step - loss: 0.1766 - val_loss: 0.1624
Epoch 7/50
302/302 [==============================] - 1115s 4s/step - loss: 0.1578 - val_loss: 0.1517
Epoch 8/50
302/302 [==============================] - 1117s 4s/step - loss: 0.1502 - val_loss: 0.1451
Epoch 9/50
302/302 [==============================] - 1117s 4s/step - loss: 0.1436 - val_loss: 0.1388
Epoch 10/50
302/302 [==============================] - 1116s 4s/step - loss: 0.137

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
print(history.history.keys())

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
test_input = np.load(files['testing'][20])

In [ ]:
decoded_data = autoencoder.predict(np.array([scaler(test_input[:,:,0])]))

In [ ]:
decoded_data.shape

In [ ]:
plt.imshow(test_input[:,:,0],cmap='hot')
plt.title('Original Input')
plt.show()
plt.imshow(inverse_scaler(decoded_data[0,:,:,0])[:,:,0],cmap='hot')
plt.title('Decoded Output')
plt.show()


In [ ]:
a = test_input[:,:,0]
b = inverse_scaler(decoded_data[0,:,:,0])[:,:,0]
mse = ((a-b)**2).mean()

In [ ]:
mse

In [ ]:
plt.imshow(a-b,cmap='hot')